In [37]:
using MuJoCo
using Plots

In [38]:
model, data = MuJoCo.sample_model_and_data()
@show typeof(model), typeof(data)

(typeof(model), typeof(data)) = (Model, Data)


(Model, Data)

In [39]:
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)

Simulation timestep: 0.005
Positions of joints: [0.0; 0.0; 1.282; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]


In [40]:
function random_controller!(m::Model, d::Data)
    nu = m.nu
    d.ctrl .= 2*rand(nu) .- 1
    return nothing
end

random_controller! (generic function with 1 method)

In [41]:
for t in 1:100
    random_controller!(model, data)
    step!(model, data)
end

In [42]:
println("New joint positions: ", data.qpos)

New joint positions: [-0.059968101431405005; -0.0325365879979836; 0.8176063539281039; 0.9876909998179103; 0.09066244730259082; -0.11955405104210155; -0.04419998198216789; 0.22261756702101268; -0.45413209243553543; -0.07515324648968187; -0.2830706820676842; -0.10001722213732842; 0.37796926675783893; -2.456008756780643; -0.6924316802685542; 0.19972648999610412; 0.11435194959168131; -0.1779221580285157; 0.03987265181507663; -2.388985882845125; -0.2814004724543829; -0.6500364690762889; 0.5050604824824727; -0.2759845265097538; -1.604528413430005; 0.0959799433922228; -1.400309521755049; 0.5427967061086939;;]


In [43]:
mj_resetData(model, data)
println("Reset joint positions: ", data.qpos)

Reset joint positions: [0.0; 0.0; 1.282; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]


In [ ]:
init_visualiser()
visualise!(model, data, controller=random_controller!)

In [44]:
reset!(model, data)
data.qpos[3] = 2
forward!(model, data) # Propagate the physics forward

In [45]:
tmax = 400
nx = model.nq + model.nv + model.na # State vector dimension
states = zeros(nx, tmax)
for t in 1:tmax
    states[:,t] = get_physics_state(model, data)
    step!(model, data)
end

In [48]:
typeof(states)

Matrix{Float64} (alias for Array{Float64, 2})

In [47]:
visualise!(model, data, trajectories = states)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [ ]:
reset!(model, data)
ctrl_states = zeros(nx, tmax)
for t in 1:tmax
    ctrl_states[:,t] = get_physics_state(model, data)
    random_controller!(model, data)
    step!(model, data)
end

In [46]:
visualise!(model, data, trajectories = [states, ctrl_states])

UndefVarError: UndefVarError: `ctrl_states` not defined

In [ ]:
model = load_model(joinpath(example_model_files_directory(), "humanoid", "humanoid.xml"))
data = init_data(model)

In [ ]:
init_visualiser()

for i in 1:3
    resetkey!(model, data, i)
    visualise!(model, data)
end

In [ ]:
# Reset to desired keyframe
keyframe = 2
resetkey!(model, data, keyframe)

# Propagate derived quantities
mj_forward(model, data)
# Set joint accelerations to 0
data.qacc .= 0

# Inspect forces from inverse dynamics
mj_inverse(model, data)
println("Required control: ", data.qfrc_inverse)

In [ ]:
# using CairoMakie
using LinearAlgebra

heights = LinRange(-0.001, 0.001, 2001) # -1mm to +1mm
# Map each height to the corresponding high force output
u_vert = map(heights) do h
    # Set model in position with qacc == 0
    resetkey!(model, data, keyframe)
    mj_forward(model, data)
    data.qacc .= 0

    # Offset the height and check required vertical forces
    data.qpos[3] += h
    mj_inverse(model, data)
    return data.qfrc_inverse[3] # 3 -> z-force
end

# Find height corresponding to minimum fictitious force (best offset)
height = heights[argmin(abs.(u_vert))]
height_mm = height*1000
heights_mm = heights .* 1000

# Compare force to weight of humanoid
weight = sum(model.body_mass) * norm(model.opt.gravity)

In [ ]:
plot(heights_mm, u_vert,label="Vertical force",size=(500,300))
plot!(heights_mm, weight*ones(length(heights)), linestyle=:dash, label="Weight")
plot!([height_mm, height_mm], [minimum(u_vert), maximum(u_vert)], linestyle=:dash)
xlabel!("Vetical offset (mm)")
ylabel!("Vertical force (N)")

In [ ]:
resetkey!(model, data, keyframe)
mj_forward(model, data)
data.qacc .= 0
data.qpos[3] += height
qpos0 = copy(data.qpos)
println(qpos0)

In [ ]:
mj_inverse(model, data)
qfrc0 = copy(data.qfrc_inverse)

M_act = data.actuator_moment
ctrl0 = pinv(M_act)' * qfrc0
println(ctrl0)

In [ ]:
data.ctrl .= ctrl0
mj_forward(model, data)
qfrc_test = data.qfrc_actuator
println("Desired forces: ", qfrc0)
println("Actual forces:  ", qfrc_test)
println("Joint forces equal? ", all((qfrc_test .≈ qfrc0)[7:end]))

In [ ]:
reset!(model, data)
data.qpos .= qpos0
data.ctrl .= ctrl0
visualise!(model, data)

In [ ]:
nu = model.nu # number of actuators/controls
nv = model.nv # number of degrees of freedom
R = Matrix{Float64}(I, nu, nu);

In [ ]:
import MuJoCo as MJ

torso = MJ.body(model, "torso")
left_foot = MJ.body(model, "foot_left")

In [ ]:
# Get Jacobian for torso CoM
reset!(model, data)
data.qpos .= qpos0
forward!(model, data)
jac_com = mj_zeros(3, nv)
mj_jacSubtreeCom(model, data, jac_com, torso.id)

# Get (left) foot Jacobian for balancing
jac_foot = mj_zeros(3, nv)
mj_jacBodyCom(model, data, jac_foot, nothing, left_foot.id)

# Design Q-matrix to balance CoM over foot
jac_diff = jac_com .- jac_foot
Qbalance = jac_diff' * jac_diff

# A cart-pole

In [28]:
model = load_model("./xmls/cartpole.xml")
data = init_data(model)

println("Initial position: ", data.qpos)
println("Initial velocity: ", data.qvel)

Initial position: [0.0; 0.0;;]
Initial velocity: [0.0; 0.0;;]


In [29]:
# Number of states and controlled inputs
nx = 2*model.nv
nu = model.nu

# Finite-difference parameters
ϵ = 1e-6
centred = true

# Compute the Jacobians
A = mj_zeros(nx, nx)
B = mj_zeros(nx, nu)
mjd_transitionFD(model, data, ϵ, centred, A, B, nothing, nothing)
@show A, B

(A, B) = ([1.0 -0.00020933539434596422 0.00999696728367769 7.112993351890879e-6; 0.0 1.0027918363759287 7.112993351890878e-6 0.009905136378663615; 0.0 -0.020933539434596422 0.9996967283677689 0.000711299335189088; 0.0 0.2791836375928739 0.0007112993351890878 0.9905136378663615], [0.0030327163223112376; -0.007112993351890878; 0.30327163223112374; -0.7112993351890878;;])


([1.0 -0.00020933539434596422 0.00999696728367769 7.112993351890879e-6; 0.0 1.0027918363759287 7.112993351890878e-6 0.009905136378663615; 0.0 -0.020933539434596422 0.9996967283677689 0.000711299335189088; 0.0 0.2791836375928739 0.0007112993351890878 0.9905136378663615], [0.0030327163223112376; -0.007112993351890878; 0.30327163223112374; -0.7112993351890878;;])

In [30]:
Q = diagm([1, 10, 1, 5]) # Weights for the state vector
R = diagm([1])           # Weights for the controls

1×1 Matrix{Int64}:
 1

In [32]:
using MatrixEquations

S = zeros(nx, nu)
_, _, K, _ = ared(A,B,R,Q,S)
@show K

K = [-0.46741668449533486 -5.5927150414501625 -0.8436677105851593 -1.4448668569004217]


1×4 Matrix{Float64}:
 -0.467417  -5.59272  -0.843668  -1.44487

In [33]:
function lqr_balance!(m::Model, d::Data)
    state = vcat(d.qpos, d.qvel)
    d.ctrl .= -K * state
    nothing
end

lqr_balance! (generic function with 1 method)

In [36]:
init_visualiser()
visualise!(model, data; controller=lqr_balance!)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
